In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage.morphology import erosion, dilation, opening, closing, white_tophat, disk
from skimage.measure import shannon_entropy
from skimage.metrics import structural_similarity as ssim

# === Métricas ===
def contraste_rms(img):
    return np.sqrt(np.mean((img - np.mean(img))**2))

def calcular_metricas(img_ref, img_proc):
    # Asegurar tipo uint8 para SSIM
    if img_proc.dtype != np.uint8:
        img_proc = cv2.normalize(img_proc, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    if img_ref.dtype != np.uint8:
        img_ref = cv2.normalize(img_ref, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    entropia = shannon_entropy(img_proc)
    contraste = contraste_rms(img_proc)
    ssim_val, _ = ssim(img_ref, img_proc, full=True)
    return entropia, contraste, ssim_val

# === Configuración ===
FILTROS_ESPACIALES = ['original', 'gaussiano', 'laplaciano', 'sobel']
FILTROS_MORFOLOGICOS = ['erosion', 'dilatacion', 'apertura', 'clausura', 'top_hat']
IMAGES_FOLDER = '.'  # Carpeta actual
RESULTS_FOLDER = './resultados'
os.makedirs(RESULTS_FOLDER, exist_ok=True)

# === Funciones de filtros espaciales ===
def aplicar_filtro_espacial(img, tipo):
    if tipo == 'original':
        return img
    elif tipo == 'gaussiano':
        return cv2.GaussianBlur(img, (5, 5), 0)
    elif tipo == 'laplaciano':
        return cv2.Laplacian(img, cv2.CV_64F)
    elif tipo == 'sobel':
        return cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=5)
    else:
        return img

# === Funciones de filtros morfológicos ===
def aplicar_filtro_morfologico(img, tipo):
    kernel = disk(3)
    if tipo == 'erosion':
        return erosion(img, kernel)
    elif tipo == 'dilatacion':
        return dilation(img, kernel)
    elif tipo == 'apertura':
        return opening(img, kernel)
    elif tipo == 'clausura':
        return closing(img, kernel)
    elif tipo == 'top_hat':
        return white_tophat(img, kernel)
    else:
        return img

# === Visualización de histogramas ===
def mostrar_histogramas(img_original, img_procesada, titulo_base):
    plt.figure()
    plt.title(f"Histograma - {titulo_base}")
    plt.xlabel("Intensidad")
    plt.ylabel("Frecuencia")
    plt.hist(img_original.ravel(), bins=256, color='blue', alpha=0.5, label='Original')
    plt.hist(img_procesada.ravel(), bins=256, color='red', alpha=0.5, label='Procesada')
    plt.legend()
    plt.savefig(os.path.join(RESULTS_FOLDER, f"{titulo_base}_hist.png"))
    plt.close()

# === Procesamiento principal ===
def procesar_imagenes():
    resumen = []

    for file in os.listdir(IMAGES_FOLDER):
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif')):
            print(f"Procesando: {file}")
            ruta = os.path.join(IMAGES_FOLDER, file)
            img = cv2.imread(ruta, cv2.IMREAD_GRAYSCALE)

            for filtro_e in FILTROS_ESPACIALES:
                img_espacial = aplicar_filtro_espacial(img, filtro_e)
                nombre_e = f"{file}_esp_{filtro_e}"
                cv2.imwrite(os.path.join(RESULTS_FOLDER, f"{nombre_e}.png"), img_espacial)
                mostrar_histogramas(img, img_espacial, nombre_e)

                # Métricas espaciales
                entropia, contraste, ssim_val = calcular_metricas(img, img_espacial)
                resumen.append([file, filtro_e, "espacial", entropia, contraste, ssim_val])

                for filtro_m in FILTROS_MORFOLOGICOS:
                    img_morf = aplicar_filtro_morfologico(img_espacial, filtro_m)
                    nombre_m = f"{file}_esp_{filtro_e}_morf_{filtro_m}"
                    cv2.imwrite(os.path.join(RESULTS_FOLDER, f"{nombre_m}.png"), img_morf)
                    mostrar_histogramas(img_espacial, img_morf, nombre_m)

                    # Métricas morfológicas
                    entropia_m, contraste_m, ssim_val_m = calcular_metricas(img_espacial, img_morf)
                    resumen.append([file, f"{filtro_e} + {filtro_m}", "espacial + morfologico", entropia_m, contraste_m, ssim_val_m])

    df = pd.DataFrame(resumen, columns=["Imagen", "Filtro aplicado", "Tipo", "Entropía", "Contraste RMS", "SSIM"])
    df.to_csv(os.path.join(RESULTS_FOLDER, "resumen_filtros.csv"), index=False)
    print("Procesamiento completado. Resultados guardados en carpeta 'resultados'.")

procesar_imagenes()

def generar_comparaciones_visuales():
    RESULTADOS_2 = './resultados2'
    os.makedirs(RESULTADOS_2, exist_ok=True)

    cuadradas = ['INSECTO', 'PROTEINA', 'SATELITAL GEOESPACIAL']
    rectangulares = ['ESPACIAL', 'SATELITAL NATURAL']

    archivos = os.listdir(RESULTS_FOLDER)

    for filtro in FILTROS_ESPACIALES:
        imgs_cuad = []
        for nombre_base in cuadradas:
            coincidencias = [f for f in archivos if nombre_base in f and f"_esp_{filtro}.png" in f]
            if coincidencias:
                img_path = os.path.join(RESULTS_FOLDER, coincidencias[0])
                img = cv2.imread(img_path)
                imgs_cuad.append(img)

        imgs_rect = []
        for nombre_base in rectangulares:
            coincidencias = [f for f in archivos if nombre_base in f and f"_esp_{filtro}.png" in f]
            if coincidencias:
                img_path = os.path.join(RESULTS_FOLDER, coincidencias[0])
                img = cv2.imread(img_path)
                imgs_rect.append(img)

        if not imgs_cuad and not imgs_rect:
            continue

        altura_cuad = min(img.shape[0] for img in imgs_cuad) if imgs_cuad else 0
        altura_rect = min(img.shape[0] for img in imgs_rect) if imgs_rect else 0

        fila1 = [cv2.resize(img, (int(img.shape[1] * altura_cuad / img.shape[0]), altura_cuad)) for img in imgs_cuad]
        fila2 = [cv2.resize(img, (int(img.shape[1] * altura_rect / img.shape[0]), altura_rect)) for img in imgs_rect]

        fila1_concat = cv2.hconcat(fila1) if fila1 else None
        fila2_concat = cv2.hconcat(fila2) if fila2 else None

        if fila1_concat is not None and fila2_concat is not None:
            ancho_max = max(fila1_concat.shape[1], fila2_concat.shape[1])
            if fila1_concat.shape[1] < ancho_max:
                pad = ancho_max - fila1_concat.shape[1]
                fila1_concat = cv2.copyMakeBorder(fila1_concat, 0, 0, 0, pad, cv2.BORDER_CONSTANT, value=[0, 0, 0])
            if fila2_concat.shape[1] < ancho_max:
                pad = ancho_max - fila2_concat.shape[1]
                fila2_concat = cv2.copyMakeBorder(fila2_concat, 0, 0, 0, pad, cv2.BORDER_CONSTANT, value=[0, 0, 0])
            comparacion = cv2.vconcat([fila1_concat, fila2_concat])
        elif fila1_concat is not None:
            comparacion = fila1_concat
        elif fila2_concat is not None:
            comparacion = fila2_concat
        else:
            continue

        output_path = os.path.join(RESULTADOS_2, f"comparacion_{filtro}.png")
        cv2.imwrite(output_path, comparacion)

generar_comparaciones_visuales()

Procesando: SATELITAL NATURAL.png


[ WARN:0@38.791] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


Procesando: SATELITAL GEOESPACIAL.png
Procesando: PROTEINA.png
Procesando: ESPACIAL.png
Procesando: INSECTO.png
Procesamiento completado. Resultados guardados en carpeta 'resultados'.
